# Machine learning

In [1]:
import numpy as np
from skimage.color import rgb2gray
import cv2
import os
import pandas as pd

from tqdm import tqdm

from matplotlib import pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Circle
%matplotlib inline

from scipy.ndimage import gaussian_filter
from scipy.ndimage import convolve
from skimage.transform import resize
from skimage.morphology import *
from skimage.feature import blob_doh
from skimage.transform import hough_circle
from skimage.feature import canny

below are the same fucntions than the main_project notebook, but doesnt output the images, only the coordinates to be processed.

In [2]:
def mask_image(gray, threshold):
    mask = gray > threshold
    return mask

In [3]:
def dilated_image(image, rep):
  dilated = dilation(image)
  for i in range(rep-1):
    dilated=dilation(dilated)
  return dilated

In [4]:
def NewdisplayCircles(image, coor, filename):
    h = image.shape[0]
    w = image.shape[1]
    # Create a figure and axis
    fig, ax = plt.subplots()
    # Read in an image and display it on the axis
    ax.imshow(image)
    # Create a circle patch with the desired coordinates and radius
    circleCoordinates = np.argwhere(coor)
    circleCoordinates = np.mean(circleCoordinates, axis=0)
    if len(circleCoordinates)==0:
        circleCoordinates=[[1,h/2,w/2]]
    r,y,x = circleCoordinates
    circle = Circle((x,y), r, color='red', fill=False)
    # Add the circle patch to the axis
    ax.add_patch(circle)
    # Save the figure
    plt.savefig(filename)
    # close the figure
    plt.close(fig)
    # read the saved file and return the image
    return plt.imread(filename)

In [5]:
def output_coor_circles(image, coor):
    circleCoordinates = np.argwhere(coor)
    circleCoordinates = np.mean(circleCoordinates, axis=0)
    return circleCoordinates 

In [6]:
def displayBlob(image, blob, filename):
    h = image.shape[0]
    w = image.shape[1]
    # Create a figure and axis
    fig, ax = plt.subplots()
    # Read in an image and display it on the axis
    ax.imshow(image)
    if len(blob)==0:
        blob=[[h/2,w/2,30]]
    blob = np.mean(blob, axis=0)
    y, x, r = blob
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    ax.add_patch(c)
    # Save the figure
    plt.savefig(filename)
    # close the figure
    plt.close(fig)
    # read the saved file and return the image
    return plt.imread(filename)

In [7]:
def output_coor_blob(image, blob):
    h = image.shape[0]
    w = image.shape[1]
    if len(blob)==0:
        blob=[[h/2,w/2,30]]
    blob = np.mean(blob, axis=0)
    y, x, r = blob
    return blob

In [8]:
path = '1_000/'
files = os.listdir(path)
num_files = len(files)

In [9]:
image = []
gray = []
list_of_names = []
for i in files:
    list_of_names.append(i)
list_of_names.sort()
for i in range(len(list_of_names)):
    image.append(imread(path+list_of_names[i]))
    gray.append(rgb2gray(image[i]))
print(len(image))
print(len(gray))

212
212


In [10]:
mask = []
mask_blob = []
disp_mask_blob = []
coor_blob = []
for i in tqdm(range(len(gray))):
    mask.append(mask_image(gray[i],0.2))
    mask_blob.append(blob_doh(mask[i], max_sigma=75, min_sigma=36, threshold=.05))
    # disp_mask_blob.append(displayBlob(image[i], mask_blob[i], "blob_estimation/gray_blob_{}.png".format(i)))
    coor_blob.append(output_coor_blob(image[i], mask_blob[i]))

100%|██████████| 212/212 [00:49<00:00,  4.29it/s]


In [11]:
column_names=['y','x','r']
df_coor_blob = pd.DataFrame(coor_blob, columns=column_names) #, columns = column_names)
df_coor_blob = df_coor_blob[['y','x']]
print(df_coor_blob.head())
print(df_coor_blob.shape)
df_coor_blob.to_csv('out.csv', index=False)

       y      x
0  305.0  330.0
1  304.0  330.0
2  304.0  329.0
3  303.0  328.0
4  300.0  326.0
(212, 2)


In [12]:
mask = []
dilated_mask = []
dilated_mask_edge = []
coor_edge_dilated_mask = []
disp_edge_dilated_mask = []
coor_circles = []

for i in tqdm(range(len(gray))):
    mask.append(mask_image(gray[i],0.2))
    dilated_mask.append(dilated_image(mask[i], 5))
    dilated_mask_edge.append(canny(dilated_mask[i], sigma=3, low_threshold=0.2, high_threshold=0.4).astype(np.uint8))
    coor_edge_dilated_mask.append(hough_circle(dilated_mask_edge[i], np.arange(15, 80)))
    # disp_edge_dilated_mask.append(NewdisplayCircles(image[i], coor_edge_dilated_mask[i], "circle_estimation/edge_dilated_mask_circle_{}.png".format(i)))
    coor_circles.append(output_coor_circles(image[i], coor_edge_dilated_mask[i]))
    


100%|██████████| 212/212 [01:51<00:00,  1.90it/s]


In [13]:
column_names = ['r','y','x']
df_coor_circles = pd.DataFrame(coor_circles, columns=column_names) #, columns = column_names)
df_coor_circles = df_coor_circles[['y','x']]
print(df_coor_circles.head())
print(df_coor_circles.shape)

            y           x
0  306.209369  331.870189
1  305.121479  332.018635
2  304.391464  331.029159
3  302.762644  329.370724
4  300.403364  327.562659
(212, 2)


In [14]:
columns_to_mean = ['y','x']
df_coor_mean = (df_coor_circles[columns_to_mean].add(df_coor_blob[columns_to_mean]))/2
print(df_coor_mean.shape)
print(df_coor_mean.head())


(212, 2)
            y           x
0  305.604684  330.935095
1  304.560740  331.009318
2  304.195732  330.014579
3  302.881322  328.685362
4  300.201682  326.781330


ground truth values are taken from the path below

In [15]:
from pathlib import Path
path = Path('SynthEyes_data/f01')
p = path.glob('**/*.pkl')
files = [x for x in p if x.is_file()]
num_files = len(files)
print(num_files)

1170


In [16]:
mean_x = []
mean_y = []
look_vec_x = []
look_vec_y = []
look_vec_z = []


for i in range(num_files):
    unpickled_df = pd.read_pickle(files[i])  
    x_coordinates = [tup[0] for tup in unpickled_df['ldmks']['ldmks_pupil_2d']]
    y_coordinates = [tup[1] for tup in unpickled_df['ldmks']['ldmks_pupil_2d']]
    mean_x.append(sum(x_coordinates) / len(x_coordinates))
    mean_y.append(sum(y_coordinates) / len(y_coordinates))
    look_vec_x.append(unpickled_df['look_vec'][0])
    look_vec_y.append(unpickled_df['look_vec'][1])
    look_vec_z.append(unpickled_df['look_vec'][2])


In [62]:
true_df = pd.DataFrame({'true_y': mean_y,'true_x': mean_x})
print(true_df.shape)
print(true_df.head())

# look_vec_df = pd.DataFrame({'vec_x': look_vec_x,'vec_y': look_vec_y,'vec_z':look_vec_z})
# print(look_vec_df.shape)
# print(look_vec_df.head())

# the y vector is depth and is not relevant in VR
look_vec_df = pd.DataFrame({'vec_x': look_vec_x,'vec_z':look_vec_z})
print(look_vec_df.shape)
print(look_vec_df.head())

(1170, 2)
      true_y     true_x
0  29.846734  54.254781
1  40.888493  44.143124
2  36.442341  43.416190
3  36.570298  66.831332
4  41.371833  73.670738
(1170, 2)
      vec_x     vec_z
0 -0.413728 -0.836536
1 -0.369369 -0.838381
2 -0.732105 -0.662866
3  0.411623 -0.827235
4  0.380012 -0.902808


In [63]:
utitlity_img_shape = [120, 80]
basic_dataset_img_shape = [640, 480]
ratio_x = basic_dataset_img_shape[0]/utitlity_img_shape[0]
ratio_y = basic_dataset_img_shape[1]/utitlity_img_shape[1]
print(ratio_x, ratio_y)

5.333333333333333 6.0


In [64]:
df_coor_circles['x'] = df_coor_circles['x']/ratio_x
df_coor_circles['y'] = df_coor_circles['y']/ratio_y

df_coor_blob['x'] = df_coor_blob['x']/ratio_x
df_coor_blob['y'] = df_coor_blob['y']/ratio_y

df_coor_mean['x'] = df_coor_mean['x']/ratio_x
df_coor_mean['y'] = df_coor_mean['y']/ratio_y

below is the machine learning part

In [65]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# define the regressor
regressor = [RandomForestRegressor(),LinearRegression(),Ridge(),Lasso(),DecisionTreeRegressor(),SVR(),MLPRegressor()]

def machine_learn(X_test,regressor, X_train=true_df, y_train=look_vec_df):
    # define the multi-output regressor
    multi_output = MultiOutputRegressor(regressor)
    # fit the model
    multi_output.fit(X_train, y_train)
    # predict
    y_pred = multi_output.predict(X_test)
    score = multi_output.score(X_test, y_pred)
    return score, y_pred

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(true_df, look_vec_df, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(936, 2)
(936, 2)
(234, 2)
(234, 2)


In [67]:
score = []
predictions = []
for i in range(len(regressor)):
    score.append(machine_learn(X_test,regressor[i], X_train, y_train)[0])
    predictions.append(machine_learn(X_test,regressor[i], X_train, y_train)[1])

In [68]:
print(predictions[0].shape)

(234, 2)


In [69]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score

def metric(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    evs = explained_variance_score(y_true, y_pred)
    metrics=[mae,mse,rmse,r2,evs]
    return metrics

on each row: the regressor tested, on each columns a different metric

In [70]:
for i in range(len(regressor)):
    print(metric(y_test, predictions[i]))

[0.052329965841705794, 0.012260364670067954, 0.11072653101252632, 0.8057813677909568, 0.806469527588797]
[0.1462587633603373, 0.03214785486595984, 0.17929822884222768, 0.34979808222123826, 0.35203257370521784]
[0.14625874258100985, 0.032147884915698755, 0.17929831264041152, 0.34979786735200097, 0.35203234422566465]
[0.1570671464856139, 0.03904484914387264, 0.1975976951886652, 0.2995583161381218, 0.30164021874273317]
[0.06275713835140435, 0.021592072741541492, 0.14694241301115718, 0.658777816702247, 0.6590482679816264]
[0.14053260067770634, 0.03064536463045801, 0.17505817498893905, 0.4461688697623289, 0.44745958130603275]
[0.1843753534862394, 0.056502163879418615, 0.2377018381910805, -0.028905925981447755, -0.012280156858281766]
